# Importing python Library

In [1]:
import mne, os, time, pickle, warnings, itertools, copy, sys, shutil
from mne.filter import filter_data as bandpass_filter
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import signal
from sklearn import svm
from scipy.signal import welch, butter, lfilter, sosfilt, sosfreqz, freqz
from scipy.integrate import simps
from scipy.stats import f_oneway
from tqdm import tqdm
from mne.preprocessing import ICA
from sklearn.decomposition import FastICA
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from skfeature.utility.construct_W import construct_W
from scipy.sparse import diags
warnings.filterwarnings('ignore')

# Global Variables

In [2]:
ch_names = ['Fp1', 'AF3', 'F3', 'F7', 'FC5', 'FC1', 'C3', 'T7', 'CP5', 'CP1', 'P3', 'P7', 'PO3',
            'O1', 'Oz', 'Pz', 'Fp2', 'AF4', 'Fz', 'F4', 'F8', 'FC6', 'FC2', 'Cz', 'C4', 'T8',
            'CP6', 'CP2', 'P4', 'P8', 'PO4', 'O2']
subject_names = ['s01', 's02', 's03', 's04', 's05', 's06', 's07', 's08', 's09', 's10', 's11', 's12', 
                 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21',
                 's22', 's23', 's24', 's25', 's26', 's27', 's28', 's29', 's30', 's31', 's32']

theta_band_range = (4, 8)   # drownsiness, emotional connection, intuition, creativity
alpha_band_range = (8, 12)  # reflection, relaxation
beta_band_range = (12, 30)  # concentration, problem solving, memory
gamma_band_range = (30, 48) # cognition, perception, learning, multi-tasking

sf = 128 # sampling frequency 128 Hz


eeg_channels = np.array(ch_names)
class_labels = ['valence', 'arousal', 'all']

my_common_path = '/Users/shyammarjit/Desktop/Brain Computer Interface/'
# deap dataset path # put the path in which deap dataset files are present
deap_dataset_path = my_common_path + 'Deap Dataset/'
# put the path location of datfiles folder s.t. subject wise folder should contain datafiles
datafiles_path = my_common_path + 'SAGA+Active Learning/band_48_fir_None_one/'
# put the path in which you want to save the csv file
save_csv_path = my_common_path + 'SAGA+Active Learning'

In [3]:
def SignalPreProcess(eeg_rawdata):
    '''
    eeg_rawdata: numpy array with the shape of (n_channels, n_samples)
    return: filtered EEG raw data of shape (n_channels, n_samples)
    '''
    assert eeg_rawdata.shape[0] == 32
    eeg_rawdata = np.array(eeg_rawdata)
  
    info = mne.create_info(ch_names = ch_names, ch_types = ['eeg' for _ in range(32)], sfreq = 128, verbose=False)
    raw_data = mne.io.RawArray(eeg_rawdata, info, verbose = False) # create MNE raw file
    # Bandpass filter of 4 Hz to 48 Hz
    raw_data.load_data(verbose = False).filter(l_freq = 4, h_freq = 48, method = 'fir', verbose = False)
    # raw_data.plot()
    
    # FAST-ICA with 31 number of components
    ica = ICA(n_components = N_C, random_state = 97, verbose = False)
    ica.fit(raw_data) # fit the data into ica
    # https://mne.tools/stable/generated/mne.preprocessing.find_eog_events.html?highlight=find_eog_#mne.preprocessing.find_eog_events
    # Take Fp1 channel as the reference channel and find the ICA score to choose artfacts score. 
    eog_indices, eog_scores = ica.find_bads_eog(raw_data.copy(), ch_name = 'Fp1', verbose = None)
    # ica.plot_scores(eog_scores)
    a = abs(eog_scores).tolist()
    if(droping_components == 'one'): # find one maximum score
        ica.exclude = [a.index(max(a))] # exclude the maximum index
            
    else: # find two maximum scores
        a_2 = a.copy()
        a.sort(reverse = True)
        exclude_index = []
        for i in range(0, 2):
            for j in range(0, len(a_2)):
                if(a[i]==a_2[j]):
                    exclude_index.append(j)
        ica.exclude = exclude_index # exclude these two maximum indeces
    ica.apply(raw_data, verbose = False) # apply the ICA 
    # common average reference
    raw_data.set_eeg_reference('average', ch_type = 'eeg')#, projection = True)
    filted_eeg_rawdata = np.array(raw_data.get_data()) # fetch the data from MNE.
    return filted_eeg_rawdata

def signal_pro(input_data):
    for i in range(input_data.shape[0]): # for each video sample call SignalPreProcess
        input_data[i] = SignalPreProcess(input_data[i].copy())
    return input_data

In [4]:
def bandpower(input_data, band):
    band = np.asarray(band)
    low, high = band # band is the tuple of (low, high)
    nperseg = (2 / low) * sf
    # Compute the modified periodogram (Welch)
    freqs, psd = welch(input_data, sf, nperseg = nperseg)
    # Find closest indices of band in frequency vector
    idx_band = np.logical_and(freqs >= low, freqs <= high)
    return np.mean(psd[idx_band]) # mean of the frequency bands

def bandpower_de(channel_index, input_data, band):
    # get Frequency wise data
    #print(input_data.shape)
    info = mne.create_info(ch_names = ch_names, ch_types = ['eeg' for _ in range(32)], sfreq = 128, verbose = False)
    raw_data = mne.io.RawArray(input_data, info, verbose = False)
    raw_data.load_data(verbose = False).filter(l_freq = band[0], h_freq = band[1], method = 'fir', verbose = False)
    filted_eeg_rawdata = np.array(raw_data.get_data()) #get the filtered data
    
    #data_band = butter_bandpass_filter(input_data, lowcut=band[0], highcut=band[1], fs = sf, order=1)
    
    std = np.std(filted_eeg_rawdata[channel_index])#, axis=1, ddof=1)
    feature = (-np.log(2*np.pi*np.e*(std)**2))/2
    return feature

In [5]:
def emotion_label(labels, class_label):
    '''
    This function gives the valence/arousal and HVHA/HVLA/LAHV/LALV class labels
    '''
    em_labels = []
    if(class_label == 'valence'):
        for i in range(0, labels.shape[0]):
            if (labels[i][0]>5): # high valence
                em_labels.append(1)
            else: # low valence
                em_labels.append(0)
        return em_labels
    elif(class_label == 'arousal'):
        for i in range(0, labels.shape[0]):
            if (labels[i][1]>5): # high arousal
                em_labels.append(1)
            else: # low arousal
                em_labels.append(0)
        return em_labels
    elif(class_label == 'all'):
        for i in range(0, labels.shape[0]):
            if (labels[i][0]>5): # high valence
                if(labels[i][1]>5): # high arousal
                    em_labels.append(1) # HVHA
                else:
                    em_labels.append(0) # HVLA
            else: # low valence
                if(labels[i][1]>5): # high arousal
                    em_labels.append(2) # LVHA
                else: # low arousal
                    em_labels.append(3) # LVLA
        return em_labels

In [6]:
def get_csv_file(subject, filter_data, labels):
    psd_theta, psd_alpha, psd_beta, psd_gamma = [], [], [], []
    psd_theta_feat, psd_alpha_feat, psd_beta_feat, psd_gamma_feat = [], [], [], []
    de_theta_feat, de_alpha_feat, de_beta_feat, de_gamma_feat = [], [], [], []
    de_theta, de_alpha, de_beta, de_gamma = [], [], [], []
    for video_no in range (len(filter_data)):
        de_input_data = copy.deepcopy(filter_data[video_no])
        for channel_no in range (0, len(filter_data[0])):
            temp = copy.deepcopy(filter_data[video_no, channel_no])
            
            # PSD Features
            psd_theta.append(bandpower(temp, theta_band_range))
            psd_alpha.append(bandpower(temp, alpha_band_range))
            psd_beta.append(bandpower(temp, beta_band_range))
            psd_gamma.append(bandpower(temp, gamma_band_range))
            
            # DE features
            de_theta.append(bandpower_de(channel_no, de_input_data, theta_band_range))
            de_alpha.append(bandpower_de(channel_no, de_input_data, alpha_band_range))
            de_beta.append(bandpower_de(channel_no, de_input_data, beta_band_range))
            de_gamma.append(bandpower_de(channel_no, de_input_data, gamma_band_range))
    # PSD feature matrix
    psd_theta = np.reshape(psd_theta, (40, 32)) # 40 videos and 32 channels theta band power
    psd_alpha = np.reshape(psd_alpha, (40, 32))
    psd_beta = np.reshape(psd_beta, (40, 32))
    psd_gamma = np.reshape(psd_gamma, (40, 32))
    
    # DE based feature matrix
    de_theta = np.reshape(de_theta, (40, 32))
    de_alpha = np.reshape(de_alpha, (40, 32))
    de_beta = np.reshape(de_beta, (40, 32))
    de_gamma = np.reshape(de_gamma, (40, 32))
    
    
    for i in range(0, len(eeg_channels)):
        psd_theta_feat.append(eeg_channels[i] + '_psd_theta')
        psd_alpha_feat.append(eeg_channels[i] + '_psd_alpha')
        psd_gamma_feat.append(eeg_channels[i] + '_psd_gamma')
        psd_beta_feat.append(eeg_channels[i] + '_psd_beta')
        
        de_theta_feat.append(eeg_channels[i] + '_de_theta')
        de_alpha_feat.append(eeg_channels[i] + '_de_alpha')
        de_gamma_feat.append(eeg_channels[i] + '_de_gamma')
        de_beta_feat.append(eeg_channels[i] + '_de_beta')
        
        
    df_psd_theta = pd.DataFrame(psd_theta, columns = psd_theta_feat)
    df_psd_alpha = pd.DataFrame(psd_alpha, columns = psd_alpha_feat)
    df_psd_beta = pd.DataFrame(psd_beta, columns =  psd_beta_feat)
    df_psd_gamma = pd.DataFrame(psd_gamma, columns = psd_gamma_feat)
    
    df_de_theta = pd.DataFrame(de_theta, columns = de_theta_feat)
    df_de_alpha = pd.DataFrame(de_alpha, columns = de_alpha_feat)
    df_de_beta = pd.DataFrame(de_beta, columns = de_beta_feat)
    df_de_gamma = pd.DataFrame(de_gamma, columns = de_gamma_feat)
    
    # make a directory to save the csv file
    new_path = newpath + subject
    try:
        os.mkdir(new_path)
    except:
        # If directory exists then delete that directory
        shutil.rmtree(new_path)
        # then make the new directory
        os.mkdir(new_path)
    #===========================   ALL Bands   ==================================
    frames = [df_psd_theta, df_psd_alpha, df_psd_beta, df_psd_gamma,\
              df_de_theta, df_de_alpha, df_de_beta, df_de_gamma]
    all_bands = pd.concat(frames, axis = 1) # join these 8 data frame columns wise, row is fixed
    #all_bands.to_csv(new_path + subject + '.csv', index = False, encoding = 'utf-8-sig')
    all_bands_valence, all_bands_arousal, all_bands_all = all_bands.copy(), all_bands.copy(), all_bands.copy()
    all_bands_valence['valence'] = emotion_label(labels, 'valence')
    all_bands_arousal['arousal'] = emotion_label(labels, 'arousal')
    all_bands_all['four_class'] = emotion_label(labels, 'all')
    all_bands_valence.to_csv(new_path + '/' + subject + '_valence.csv', index = False, encoding = 'utf-8-sig')
    all_bands_arousal.to_csv(new_path + '/' + subject + '_arousal.csv', index = False, encoding = 'utf-8-sig')
    all_bands_all.to_csv(new_path + '/' + subject + '_four_class.csv', index = False, encoding = 'utf-8-sig')

In [ ]:
N_C = None
droping_components = 'one'
valence_acc, arousal_acc, four_acc = [], [], []
for subject in subject_names:
    newpath = save_csv_path + '/band_48_fir_None_one/'
    try:
        # If the directory already exists then don't make any new dirctory
        os.mkdir(newpath)
    except:
        pass
    # load the dataset
    with open(deap_dataset_path + subject + '.dat', 'rb') as f:
        raw_data = pickle.load(f, encoding = 'latin1')
    # raw_data has two key 'data' and 'labels'
    data = raw_data['data']
    labels = raw_data['labels']
    # we are excluding 3s pre base line i.e. first 3*128 = 384 data points from time series data
    reduced_eeg_data  = data[0:40, 0:32, 384:8064]
    filter_data = signal_pro(reduced_eeg_data.copy())
    get_csv_file(subject, filter_data, labels)
    print('Done: ', subject)

Done:  s01
Done:  s02
Done:  s03
Done:  s04
Done:  s05
Done:  s06
Done:  s07
Done:  s08
Done:  s09
Done:  s10
Done:  s11
Done:  s12
Done:  s13
Done:  s14
Done:  s15
Done:  s16
Done:  s17
Done:  s18
Done:  s19
Done:  s20
Done:  s21
Done:  s22
Done:  s23
Done:  s24
Done:  s25
Done:  s26
Done:  s27
Done:  s28
Done:  s29


In [8]:
N_C = None
droping_components = 'one'
valence_acc, arousal_acc, four_acc = [], [], []
for subject in subject_names[29:]:
    newpath = save_csv_path + '/band_48_fir_None_one/'
    try:
        # If the directory already exists then don't make any new dirctory
        os.mkdir(newpath)
    except:
        pass
    # load the dataset
    with open(deap_dataset_path + subject + '.dat', 'rb') as f:
        raw_data = pickle.load(f, encoding = 'latin1')
    # raw_data has two key 'data' and 'labels'
    data = raw_data['data']
    labels = raw_data['labels']
    # we are excluding 3s pre base line i.e. first 3*128 = 384 data points from time series data
    reduced_eeg_data  = data[0:40, 0:32, 384:8064]
    filter_data = signal_pro(reduced_eeg_data.copy())
    get_csv_file(subject, filter_data, labels)
    print('Done: ', subject)

Done:  s30
Done:  s31
Done:  s32
